## Uproszczony schemat oleju i octu

Niech $M = (m_{1}, ..., m_{k})$ to będą wiadomości złożone z $k$ elementów wybranych ze skończonego pola $F$ poziomu $q$ (pola Galois poziomu $q$)

Niech $X = (x_{1}, ..., x_{2k})$ składa się z $2k$ elementów, należących do pola $GF(q)$. Wektor $X$ jest prawidłową sygnaturą $M$, jeżeli $G(x) = M$, gdzie $G(X) : GF(q)^{2k} \rightarrow GF(q)^{k}$. Zatem mapowanie $G$ można zapisać jako $G(X) = (G_{1}(X), G_{2}(X), ..., G_{k}(X))$, gdzie $G_{k}(X)$ można zapisać jako $G(X) = \sum_{i}\sum_{j} c_{ij}x_{i}x_{j}$. Widać zatem, że $G_{e}(X) = X^{T} G_{e} X$, gdzie $G_{e}$ to macierz diagonalna $2k$ współczynników.

Każda wiadomość $M$ ma około $qk$ możliwych podpisów, ale znalezienie jakiegokolwiek z nich jest trudne ze względu na nieliniowość równania. Prawdziwy podpisujący może rozwiązać dane równania i obliczyć $X$ wykorzystując strukturę $G$, zdefiniowaną daną konstrukcją:

Niech $A$ będzie losowo wybraną nieosobliwą ($det A \ne 0$) macierzą $2k\times2k$ nad polem $GF(q)$ oraz niech $Y=(y_{1}, ..., y_{2k})$ stanowi $Y=AX$. Niech $F=(F_{1}, ..., F_{k})$ będzie tensorem losowych $k$ macierzy rozmiaru $2k\times2k$ takich, że:

$$
    F_{e} = \begin{pmatrix}
0_{k} & B_{1} \\
B_{2} & B_{3}
\end{pmatrix}
$$

Gdzie $0_{k}$ to macierz zer rozmiaru $k \times k$. Wyprowadzając $F_{e}(Y) = Y^{T} F_{e} Y$. W takim wypadku podstawiając $Y=AX$ otrzymujemy $F_{e}(X) = X^{T} A^{T} F_{e} A X$. Forma ta jest podobna do formy $G(X)$. Kluczem weryfikacyjnym staje się zatem $A^{T} F_{e} A$.

Ukrytym kluczem podpisującego staję się macierz $A$, która wykonuje translacje $X$ do $Y$. Na poziomie zmiennej $Y$ formą kwadratową jest równanie $Y^{T} F_{e} Y$. Fakt, że lewy górny minor $F_{e}$ jest macierzą zerową, oznacza, że dla formy $c_{ij}y_{i}y_{j}$ maksymalnie jedna wartość $i, j$ może być w zakresie $[1, k]$, a więc wszystkie zmienne z pierwszej połowy wektora $Y$ (nazywanych także olejem) pojawiają się liniowo w formie kwadratowej, gdzie wszystkie zmienne z drugiej połowy $Y$ (nazywanych także octem) mogą pojawić się albo liniowo, albo kwadratowo w formach kwadratowych

$$
    R = Y^{T} F Y \implies R_{i} = \sum_{j=0}^{2k} y_{i}y_{j}f_{ij} = \sum_{j=k}^{2k} y_{i}y_{j} f_{ij}
$$

Podpisanie:
* Przypisz losową wartość octowi $y_{>k} = 0$
* Uprość równanie kwadratowe, definiowane przez $Y^{T} F_{e} Y = m_{e}$. Wynikowe równanie jest liniowe i zawiera tylko olej
* Rozwiąż $k$ systemów układów liniowych w $k$ zmiennych. Jeżeli macierze są osobliwę, wylosuj inne liczby i powróć do kroku 1.
* Dokonaj mapowania $X=A^{-1}Y$
* $X$ jest podpisem $M$


Atakujący musi znać $2k$ zmiennych losowych

### Algorytm

In [1]:
import galois as gf
import numpy as np

from dataclasses import dataclass

FIELD = gf.GF(2 ** 3)

def random_nonsingular_matrix(k: int) -> gf.FieldArray:
    while True:
        matrix = FIELD.Random((k, k))
        if abs(np.linalg.det(matrix)) > 1e-3:
            return matrix


def make_matrix_f(k: int) ->  gf.FieldArray:
    return np.concatenate(
        (np.concatenate((FIELD.Zeros((k, k, k)), FIELD.Random((k, k, k))), axis=2),
        np.concatenate((FIELD.Random((k, k, k)), FIELD.Random((k, k, k))), axis=2)),
        axis=1
    )


@dataclass(frozen=True)
class PublicKey:
    key: gf.FieldArray

    def unsign(self, encrypted: gf.FieldArray) -> gf.FieldArray:
        k = self.key.shape[0]
        assert k * 2 == encrypted.shape[0]
        decrypted = encrypted.reshape((1, 1, 2 * k)) @ self.key @ encrypted.reshape((1, 2 * k, 1))
        assert decrypted.shape[1] == 1
        assert decrypted.shape[2] == 1
        return decrypted[:, 0, 0]

    def validate(self, data: gf.FieldArray, cert: gf.FieldArray) -> bool:
        decrypted = self.unsign(cert)
        return np.equal(decrypted, data)


@dataclass(frozen=True)
class SecretKey:
    A: gf.FieldArray
    F: gf.FieldArray

    @classmethod
    def make_key(cls, k: int) -> "SecretKey":
        return cls(A=random_nonsingular_matrix(2 * k), F=make_matrix_f(k))

    def sign(self, data: gf.FieldArray, repeats: int = 10) -> gf.FieldArray:
        k = data.shape[0]
        assert self.A.shape[0] == k * 2

        for _ in range(repeats):
            vinegar = FIELD.Random((k))

            # Y = [O, V]
            #                [0,  B1] |O|
            # m_e = [O, V] * [B2, B3] |V|

            #                |B1V|
            # m_e = [O, V] * |B2O + B3V|

            # m_e = O^{T} B1V + V^{T}B2O + V^{T}B3V
            # dla skalarów transpozycja nie ma znaczenia, więc O^{T} B1V = (O^{T}B1V)^{T} = V^{T}B1{^T} O
            # m_e = V^{T}B1{^T}O + V^{T}B2O + V^{T}B3V
            # m_e - V^{T}B3V = (V^{T}B1{^T} + V^{T}B2)O
            # M = L * O
            # Co dla wszystkich e daje równanie

            b1 = self.F[:, :k, k:]
            b2 = self.F[:, k:, :k]
            b3 = self.F[:, k:, k:]

            matrix = vinegar.reshape((1, 1, k)) @ (b1.swapaxes(1, 2) + b2)
            assert matrix.shape[1] == 1

            matrix = matrix[:, 0, :] # usuń nieistotne wymiary jednościowe
            left_mat = vinegar.reshape((1, 1, k)) @ b3 @ vinegar.reshape((1, k, 1))
            assert left_mat.shape[1] == 1
            assert left_mat.shape[2] == 1
            left = data - left_mat[:, 0, 0] # usuń nieistotne wymiary jednościowe

            # left = matrix * O
            # O = matrix^{-1} * left
            try:
                oil = np.linalg.solve(matrix, left)
            except np.linalg.LinAlgError:
                continue # Macierz jest osobliwa, spróbujmy ponownie

            y = np.concatenate((oil, vinegar), axis=0).reshape(2 * k, 1)

            return (np.linalg.inv(self.A) @ y)[:, 0]


        raise ValueError(f'Tried to sign data {repeats} times but without success')

    @property
    def public_key(self) -> PublicKey:
        return PublicKey(key=self.A.T @ self.F @ self.A)


key = SecretKey.make_key(2)

original = FIELD.Random((2))
cert = key.sign(original)

public = key.public_key

print("Original: ", original)
print("Cert: ", cert)

decrypted = public.unsign(cert)
print("Decrypted: ", decrypted)

Original:  [7 5]
Cert:  [2 7 6 7]
Decrypted:  [7 5]


## Atak linearyzujący

Na początku musimy założyć, że każda macierz $G_{e} = A^{T}F_{e}A$ jest nieosobliwa. Jeżeli jest osobliwa, to nie jest ona brana dalej pod uwagę. Następnie konieczne jest zdefiniowanie zbioru $T$, który stanowi domknięcie zbioru wszystkich macierzy $G_{ij}=G^{-1}_{i}G_{j}$ nad operacją dodawania, mnożenia oraz mnożenia przez stałą z $GF(k)$.

In [3]:
# Scipy posiada wygodną funkcję do usuwania liniowo zależnych wierszy z macierzy,
# ale niestety nie wspiera ona operacji na polach Galois. Z tego powodu problematyczne
# funkcje zostały zmockowane ich poprawdzie dzałającymi wariantami, co pozwala
# na prawidłowe użycie funkcji
from scipy.optimize import _remove_redundancy
from scipy.linalg import lu_solve

_remove_redundancy.np.allclose = lambda a, b: np.all(a, FIELD(b))

def plu_decompose(x: gf.FieldArray) -> tuple[tuple[gf.FieldArray, gf.FieldArray], gf.FieldArray]:
    p, l, u = x.plu_decompose()
    return (l, u), p

_remove_redundancy.scipy.linalg.lu_factor = plu_decompose
# _remove_redundancy.scipy.linalg.lu_solve


In [2]:
from itertools import product
from typing import Generator

class Closure:
    def __init__(self, key: PublicKey) -> None:
        self.__non_singular = list(filter(lambda x: abs(np.linalg.det(x)) > 1e-8, key.key))

    # @classmethod
    # def __remove_lineary_dependent(cls, data: list[gf.FieldArray]) -> list[gf.FieldArray]:
    #     flattened = np.stack(list(map(gf.FieldArray.flatten, data)), axis=0)
    #     result, *_ = _remove_redundancy._remove_redundancy_pivot_dense(flattened, FIELD.Zeros(len(flattened)))
    #     return [matrix.reshape(data[0].shape) for matrix in result]

    def __iter__(self) -> Generator[gf.FieldArray]:
        yield from [np.linalg.inv(gi) @ gj for gi, gj in product(self.__non_singular, self.__non_singular)]
        # yield from (generator := self.__remove_lineary_dependent(generator))
        # Jeżeli tyle nie wystarczy, to zacznijmy przetwarzać iloczyny

        # while True:
        #     old_gen_len = len(generator)
        #     new_hope = [a * b for a, b in product(generator, generator)]
        #     generator = self.__remove_lineary_dependent(generator + new_hope)
        #     if len(generator) == old_gen_len:
        #         return
        #     yield from generator[old_gen_len:]

In [3]:
print(len(list(Closure(public))))

4


## Bazowe

$T_{1}R_{1}$

$$
    ( T_{1} R, T_{2}R, T_{3}R, ..., T_{n}R)
$$

$$
    \begin{pmatrix}
    \sum_{i}T^{1}_{0, i}r_{i} & \sum_{i}T^{2}_{0, i}r_{i} & ... & \sum_{i}T^{n}_{0, i}r_{i}\\
    \sum_{i}T^{1}_{1, i}r_{i} & \sum_{i}T^{2}_{1, i}r_{i} & ... & \sum_{i}T^{n}_{1, i}r_{i} \\
    ... & ... & ... & ... \\
    \sum_{i}T^{1}_{2k, i}r_{i} & \sum_{i}T^{2}_{2k, i}r_{i} & ... & \sum_{i}T^{n}_{2k, i}r_{i} \\
    \end{pmatrix}
$$

$$ \left( M_{k+1} = \sum_{k} s_{k} M_{i} \right)$$

$$ 
    M_{k} = \begin{pmatrix}
    \sum_{i}T^{1}_{k, i}r_{i} & \sum_{i}T^{2}_{k, i}r_{i} & ... & \sum_{i}T^{N}_{k, i}r_{i} \\
    \end{pmatrix}
$$

$$ 
    M_{k+1} = \begin{pmatrix}
    \sum_{j}\sum_{k}T^{1}_{k, i}r_{i}s_{k} & \sum_{j}\sum_{k}T^{2}_{k, i}r_{i}s_{k} & ... & \sum_{j}\sum_{k}T^{N}_{k, i}r_{i}s_{k} \\
    \end{pmatrix}
$$

$$ 
   \begin{pmatrix}
    \sum_{i}T^{1}_{k+1, i}r_{i} & \sum_{i}T^{2}_{k+1, i}r_{i} & ... & \sum_{i}T^{N}_{k+1, i}r_{i} \\
    \end{pmatrix} = \begin{pmatrix}
    \sum_{j}\sum_{i}T^{1}_{k, i}r_{i}s_{j} & \sum_{j}\sum_{i}T^{2}_{k, i}r_{i}s_{j} & ... & \sum_{j}\sum_{i}T^{N}_{k, i}r_{i}s_{j} \\
    \end{pmatrix}
$$


$$
    \sum_{i}T^{1}_{k+1, i}r_{i} = \sum_{i}\sum_{j} T^{1}_{j} r_{i}s_{j}
$$
$$
    \sum_{i}T^{p}_{k+1, i}r_{i} = \sum_{i}\sum_{j} T^{p}_{j} r_{i}s_{j}
$$

Lineralyzacja

$$
\sum_{i}T^{p}_{k+1, i}r_{i} = \sum_{i}\sum_{j} T^{p}_{j} z_{ij}
$$

## Naiwna

$$
    c_{p} = \sum_{i}r_{i} \sum_{k} T^{p}_{k, i}  s_{k}

$$

$$    
    c_{p} = S^{T} T^{p} R
$$

$$    
    c_{p} = R^{T}T^{T}_{p} S 
$$

$$    
    C = O S 
$$

$$    
    O^{-1}C = S 
$$

In [4]:
def naive_linear(public: PublicKey, count: int = 5000) -> tuple[gf.FieldArray, gf.FieldArray]:
    for _ in range(count):
        k2 = public.key.shape[-1]
        k = k2 // 2
        r = FIELD.Random((k2))
        closure = np.stack(list(Closure(public)), axis=0)
        n = closure.shape[0]

        C = closure[:, k + 1, :].reshape((n, k2)) @ r.reshape((k2, 1))
        C = C.reshape((n))

        lhs = r.reshape((1, 1, k2)) @ closure[:, :k, :].swapaxes(1, 2)
        lhs = lhs.reshape((n, k))
        sublhs = lhs[:k, :]
        suplhs = lhs[k:, :]

        try:
            S = np.linalg.inv(sublhs) @ C[:k]
        except np.linalg.LinAlgError:
            continue

        if np.all(suplhs @ S == C[k:]):
            return r, S

        if _ % 20 == 19:
            print(_)
    raise ValueError("Cannot find a solution")


r, s = naive_linear(public)

print(r, s)

[7 4 0 0] [4 7]


In [385]:
def make_oil_base(public: PublicKey, count: int = 50) -> gf.FieldArray:
    k = public.key.shape[-1] // 2
    for _ in range(count):
        base = []
        for i in range(k):
            base.append(FIELD(np.array([0] * (k + i) + [1] + [0] * (k - i - 1))))
        for _ in range(k):
            base.append(naive_linear(public)[0])
        base = np.stack(base, axis=0)
        if np.linalg.det(base) != 0:
            return base


In [386]:
oil_base = make_oil_base(public)
print(oil_base)

[[0 0 1 0]
 [0 0 0 1]
 [6 5 7 7]
 [4 1 7 2]]


In [489]:
def forge_key(oil_base: gf.FieldArray, public: PublicKey) -> gf.FieldArray:
    return oil_base.T @ public.key @ oil_base

forged = forge_key(oil_base, public)
print("Forged: ", forged)

forged_private = SecretKey(A=oil_base, F=forged)

data = FIELD(np.array([0, 4]))
forged_cert = forged_private.sign(data)

print(f"Data: {data}")
print(f"Forged: {forged_cert}")

print(f"Verify: {public.unsign(forged_cert)}")


Forged:  [[[5 5 7 0]
  [2 4 3 4]
  [4 0 5 2]
  [3 1 3 0]]

 [[5 4 3 1]
  [3 1 7 6]
  [2 4 3 6]
  [2 1 7 6]]]
Data: [0 4]
Forged: [5 2 2 3]
Verify: [0 4]


In [1]:
def kernel(matrix: gf.FieldArray) -> gf.FieldArray:
    return np.linalg.solve(matrix, FIELD.Zeros((matrix.shape[-1], 1)))

def characteristic_poly_2(public: PublicKey, count: int = 10000) -> gf.FieldArray:
    for _ in range(count):
        g_i = FIELD.Zeros(public.key.shape[1:])
        g_j = FIELD.Zeros(public.key.shape[1:])

        while abs(np.linalg.det(g_i)) < 1e-8:
            g_i = FIELD.Zeros(g_i.shape)
            for g in public.key:
                g_i += FIELD.Random((1)) * g

        while abs(np.linalg.det(g_j)) < 1e-8:
            g_j = FIELD.Zeros(g_j.shape)
            for g in public.key:
                g_j += FIELD.Random((1)) * g

        g_ij = np.linalg.inv(g_i) * g_j

        polys, coeffs = g_ij.characteristic_poly().factors()
        if len(coeffs) == 2:
            first_kernel = polys[0](g_ij).null_space()
            second_kernel = polys[1](g_ij).null_space()
            if first_kernel.shape[0] == public.key.shape[-1] // 2:
                return first_kernel
            if second_kernel.shape[0] == public.key.shape[-1] // 2:
                return second_kernel
        print(_, len(coeffs), polys, coeffs)
    raise ValueError("Cannot find a solution")

In [50]:
characteristic_poly_2(public)

0 1 [Poly(x^8 + x^7 + 2x^6 + x^5 + 3x^4 + 7x^2 + x + 3, GF(2^3))] [1]
1 3 [Poly(x^2 + 5x + 7, GF(2^3)), Poly(x^3 + 4x^2 + 4x + 3, GF(2^3)), Poly(x^3 + 7x^2 + 4x + 4, GF(2^3))] [1, 1, 1]
2 2 [Poly(x^4 + 2x^3 + 5x^2 + 6x + 6, GF(2^3)), Poly(x^4 + 5x^3 + 5x^2 + 3x + 5, GF(2^3))] [1, 1]
3 1 [Poly(x^8 + 5x^7 + 4x^6 + 6x^5 + x^4 + 5x^3 + 2x^2 + 3x + 2, GF(2^3))] [1]
4 4 [Poly(x, GF(2^3)), Poly(x + 4, GF(2^3)), Poly(x + 7, GF(2^3)), Poly(x^5 + 6x^4 + 4x^3 + 6x^2 + 4x + 5, GF(2^3))] [1, 1, 1, 1]
5 1 [Poly(x^8 + x^7 + 7x^5 + 3x^4 + 4x^3 + 7x^2 + x + 2, GF(2^3))] [1]
6 2 [Poly(x^2 + 5x + 3, GF(2^3)), Poly(x^6 + x^5 + 5x^4 + 3x^3 + 7x^2 + x + 3, GF(2^3))] [1, 1]
7 3 [Poly(x + 7, GF(2^3)), Poly(x^2 + 6x + 6, GF(2^3)), Poly(x^5 + 4x^4 + 6x^3 + 5, GF(2^3))] [1, 1, 1]
8 2 [Poly(x + 7, GF(2^3)), Poly(x^6 + 5x^5 + 5x^4 + 2x^3 + 4x^2 + x + 1, GF(2^3))] [2, 1]
9 3 [Poly(x + 7, GF(2^3)), Poly(x^2 + 6x + 5, GF(2^3)), Poly(x^3 + 2x^2 + 5x + 3, GF(2^3))] [3, 1, 1]
10 3 [Poly(x + 5, GF(2^3)), Poly(x^2 + 6x + 

KeyboardInterrupt: 